# ДЗ №7: двухвыборочные гипотезы (медианы и распределения) + бутстрап-проверка

Датасет: **MovieLens 10M** (GroupLens) — логи взаимодействий пользователей (оценки фильмов во времени).  
Мы заново загружаем данные, готовим признаки, сегментируем аудиторию и проверяем 4 гипотезы + 5-й пункт (бутстрап).

**Метрики (как в прошлом ДЗ):**
- `mean_rating` — средняя оценка пользователя
- `median_rating` — медианная оценка пользователя
- `share_high` — доля оценок ≥ 4
- `unique_movies` — разнообразие (кол-во уникальных фильмов)

> Но для требований текущего ДЗ нам достаточно выбрать **1 непрерывную** метрику и **1 дискретную** (если дискретной не было — создать).  
> Здесь: непрерывная = `mean_rating`, дискретная = `unique_movies`.


**0. Импорт и настройки**


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

RANDOM_STATE = 42
rng = np.random.default_rng(RANDOM_STATE)


**1. Загрузка логов (MovieLens 10M) и сэмплирование событий**

В Colab удобно сразу брать подвыборку строк, чтобы не падать по памяти.


In [ ]:
import io, zipfile, requests

MOVIELENS_URL = "https://files.grouplens.org/datasets/movielens/ml-10m.zip"

resp = requests.get(MOVIELENS_URL, stream=True)
resp.raise_for_status()

z = zipfile.ZipFile(io.BytesIO(resp.content))
names = z.namelist()

ratings_path = [p for p in names if p.endswith("/ratings.dat")][0]

with z.open(ratings_path) as f:
    ratings = pd.read_csv(
        f,
        sep="::",
        engine="python",
        header=None,
        names=["userId", "movieId", "rating", "timestamp"],
        dtype={"userId":"int32","movieId":"int32","rating":"float32","timestamp":"int64"},
    )

# Сэмпл событий (регулируй при необходимости)
N_EVENTS = 1_000_000
ratings = ratings.sample(n=N_EVENTS, random_state=RANDOM_STATE).reset_index(drop=True)

ratings.head(), ratings.shape


**2. Подготовка пользовательских метрик (user-level)**

Считаем метрики на уровне пользователя, чтобы потом сравнивать сегменты статистическими тестами.


In [ ]:
user = ratings.groupby("userId", sort=False).agg(
    n_events=("movieId", "size"),
    mean_rating=("rating", "mean"),
    median_rating=("rating", "median"),
    share_high=("rating", lambda x: float(np.mean(x >= 4.0))),
    unique_movies=("movieId", "nunique"),
).reset_index()

user.head(), user.shape


**3. Сегментация аудитории**

Разобьем пользователей на 3 сегмента по активности (`n_events`) — tertiles:
- Light (мало событий)
- Medium
- Heavy (много событий)


In [ ]:
q1, q2 = user["n_events"].quantile([1/3, 2/3]).to_list()

def segment(n):
    if n <= q1:
        return "Light"
    elif n <= q2:
        return "Medium"
    return "Heavy"

user["segment"] = user["n_events"].apply(segment)
user["segment"].value_counts()


**4. Выбор двух групп для двухвыборочных гипотез**

Будем сравнивать наиболее контрастные сегменты: **Light vs Heavy**.


In [ ]:
A, B = "Light", "Heavy"
dfA = user[user["segment"] == A].copy()
dfB = user[user["segment"] == B].copy()

print("Users in A:", len(dfA), "Users in B:", len(dfB))
dfA[["n_events","mean_rating","unique_movies"]].head()


---

## **Пункт 1. Сформировать 4 двухвыборочные гипотезы (медианы и распределения) для дискретного и непрерывного случая**

Выбираем:
- **Непрерывная** метрика: `mean_rating`
- **Дискретная** метрика: `unique_movies`

**(1) Непрерывная, медианы (`mean_rating`)**  
- H0: `median(mean_rating)_Light = median(mean_rating)_Heavy`  
- H1: медианы различаются

**(2) Непрерывная, распределения (`mean_rating`)**  
- H0: распределения `mean_rating` одинаковы в Light и Heavy  
- H1: распределения различаются

**(3) Дискретная, медианы (`unique_movies`)**  
- H0: `median(unique_movies)_Light = median(unique_movies)_Heavy`  
- H1: медианы различаются

**(4) Дискретная, распределения (`unique_movies`)**  
- H0: распределения `unique_movies` одинаковы в Light и Heavy  
- H1: распределения различаются


---

## **Пункт 2. Проверить каждую гипотезу релевантным тестом + обосновать выбор**

**Почему именно такие тесты:**
- Для **медиан** используем **Mood’s median test** (`scipy.stats.median_test`): это прямой тест равенства медиан, подходит для 2 независимых выборок, не требует нормальности.
- Для **распределений**:
  - непрерывное: **KS 2-sample** (Kolmogorov–Smirnov), сравнивает CDF и проверяет равенство распределений целиком;
  - дискретное: **χ² тест однородности** по таблице частот (для устойчивости используем биннинг).


### **2.1. Тест для гипотезы (1): медианы `mean_rating`**
Тест: **Mood’s median test**


In [ ]:
x = dfA["mean_rating"].dropna().to_numpy()
y = dfB["mean_rating"].dropna().to_numpy()

med_stat, med_p, grand_median, table = stats.median_test(x, y, ties="below")

{
    "metric": "mean_rating",
    "median_A": float(np.median(x)),
    "median_B": float(np.median(y)),
    "grand_median": float(grand_median),
    "p_value": float(med_p),
    "table_2x2": table.tolist()
}


### **2.2. Тест для гипотезы (2): распределения `mean_rating`**
Тест: **KS 2-sample**


In [ ]:
ks_cont = stats.ks_2samp(x, y, alternative="two-sided", method="asymp")
{"metric": "mean_rating", "KS_stat": float(ks_cont.statistic), "p_value": float(ks_cont.pvalue)}


### **2.3. Тест для гипотезы (3): медианы `unique_movies`**
Тест: **Mood’s median test**


In [ ]:
x_d = dfA["unique_movies"].dropna().to_numpy()
y_d = dfB["unique_movies"].dropna().to_numpy()

med_stat_d, med_p_d, grand_median_d, table_d = stats.median_test(x_d, y_d, ties="below")

{
    "metric": "unique_movies",
    "median_A": float(np.median(x_d)),
    "median_B": float(np.median(y_d)),
    "grand_median": float(grand_median_d),
    "p_value": float(med_p_d),
    "table_2x2": table_d.tolist()
}


### **2.4. Тест для гипотезы (4): распределения `unique_movies`**
Тест: **χ² тест однородности** по бинированным значениям (частоты по категориям)


In [ ]:
# Биннинг, чтобы не было слишком редких категорий (и чтобы expected частоты были разумные)
bins = [0, 5, 10, 20, 40, 80, 160, np.inf]
labels = ["1-5","6-10","11-20","21-40","41-80","81-160","160+"]

A_cat = pd.cut(dfA["unique_movies"], bins=bins, labels=labels, include_lowest=True).astype("object")
B_cat = pd.cut(dfB["unique_movies"], bins=bins, labels=labels, include_lowest=True).astype("object")

ct = pd.crosstab(
    pd.Series([A]*len(A_cat) + [B]*len(B_cat), name="group"),
    pd.Series(pd.concat([A_cat, B_cat], ignore_index=True), name="unique_movies_bin")
)

ct


In [ ]:
chi2_stat, chi2_p, dof, expected = stats.chi2_contingency(ct.values)

{
    "metric": "unique_movies (binned)",
    "chi2": float(chi2_stat),
    "dof": int(dof),
    "p_value": float(chi2_p),
    "min_expected": float(np.min(expected)),
}


---

## **Пункт 2 (5-й пункт). Проверка результатов с бутстрапом**

Сделаем “бутстрап под H0” (null bootstrap):  
- объединяем две группы в один pool (это модель H0: группы из одного распределения),  
- **семплируем с возвращением** две выборки размеров `len(A)` и `len(B)`,  
- считаем статистику (разность медиан / KS / χ²),  
- p-value = доля бутстрап-статистик, которые по модулю ≥ наблюдаемой.

Это дает сравнимую с тестами проверку без асимптотических допущений (но вычислительно дороже).


In [ ]:
def null_bootstrap_pvalue(x, y, stat_fn, B=400, seed=42):
    rng = np.random.default_rng(seed)
    x = np.asarray(x); y = np.asarray(y)
    n_x, n_y = len(x), len(y)
    pooled = np.concatenate([x, y])

    obs = float(stat_fn(x, y))
    boot = np.empty(B, dtype=float)
    for b in range(B):
        xb = rng.choice(pooled, size=n_x, replace=True)
        yb = rng.choice(pooled, size=n_y, replace=True)
        boot[b] = stat_fn(xb, yb)

    p = (np.sum(np.abs(boot) >= abs(obs)) + 1) / (B + 1)  # two-sided
    return obs, float(p), boot

# Статистика 1: разность медиан
stat_median_diff = lambda a,b: np.median(a) - np.median(b)

# Статистика 2: KS
stat_ks = lambda a,b: stats.ks_2samp(a, b, alternative="two-sided", method="asymp").statistic

# Статистика 3: chi2 по бинам
def stat_chi2_bins(a, b):
    a_cat = pd.cut(a, bins=bins, labels=labels, include_lowest=True)
    b_cat = pd.cut(b, bins=bins, labels=labels, include_lowest=True)
    bins_series = pd.concat([pd.Series(a_cat), pd.Series(b_cat)], ignore_index=True)
    ct_local = pd.crosstab(
        pd.Series([0]*len(a_cat) + [1]*len(b_cat), name='group'),
        pd.Series(bins_series, name='bin')
    )
    # Выкидываем бины, которые отсутствуют в обеих группах (иначе ожидаемые частоты могут стать 0)
    col_sums = ct_local.sum(axis=0)
    ct_local = ct_local.loc[:, col_sums > 0]
    # Если после фильтрации осталась одна колонка, статистика = 0 (распределения тривиально одинаковы по бинам)
    if ct_local.shape[1] <= 1:
        return 0.0
    chi2_stat, _, _, expected = stats.chi2_contingency(ct_local.values)
    # На всякий случай: если всё равно есть нули в expected (редко), добавим маленький псевдосчет
    if np.any(expected == 0):
        ct_eps = ct_local.values.astype(float) + 1e-6
        chi2_stat = stats.chi2_contingency(ct_eps)[0]
    return float(chi2_stat)

B_BOOT = 300  # держим маленьким для Colab

# (1) mean_rating median
obs_mr_med, p_mr_med, _ = null_bootstrap_pvalue(x, y, stat_median_diff, B=B_BOOT, seed=RANDOM_STATE)

# (2) mean_rating distr (KS)
obs_mr_ks, p_mr_ks, _ = null_bootstrap_pvalue(x, y, stat_ks, B=B_BOOT, seed=RANDOM_STATE)

# (3) unique_movies median
obs_um_med, p_um_med, _ = null_bootstrap_pvalue(x_d, y_d, stat_median_diff, B=B_BOOT, seed=RANDOM_STATE)

# (4) unique_movies distr (chi2)
obs_um_chi2, p_um_chi2, _ = null_bootstrap_pvalue(x_d, y_d, stat_chi2_bins, B=B_BOOT, seed=RANDOM_STATE)

bootstrap_results = pd.DataFrame([
    ["Median(mean_rating)", obs_mr_med, p_mr_med],
    ["Distr(mean_rating) KS", obs_mr_ks, p_mr_ks],
    ["Median(unique_movies)", obs_um_med, p_um_med],
    ["Distr(unique_movies) chi2", obs_um_chi2, p_um_chi2],
], columns=["Hypothesis","obs_stat","bootstrap_pvalue"])

bootstrap_results


---

## **Пункт 3. Сравнение результатов (пункты 2.1–2.4) с бутстрапом и объяснение различий**

Соберем p-value классических тестов и бутстрапа в одну таблицу.


In [ ]:
alpha = 0.05

classic_results = pd.DataFrame([
    ["Median(mean_rating)", float(med_p)],
    ["Distr(mean_rating) KS", float(ks_cont.pvalue)],
    ["Median(unique_movies)", float(med_p_d)],
    ["Distr(unique_movies) chi2", float(chi2_p)],
], columns=["Hypothesis","p_value_test"])

out = classic_results.merge(bootstrap_results[["Hypothesis","bootstrap_pvalue"]], on="Hypothesis", how="left")
out["reject_test_0.05"] = out["p_value_test"] < alpha
out["reject_bootstrap_0.05"] = out["bootstrap_pvalue"] < alpha
out


**Как интерпретировать расхождения (если они есть):**
- Бутстрап здесь “под H0” и численно приближает нулевое распределение статистики; при больших выборках результаты обычно совпадают с асимптотическими тестами.
- Если отличаются:
  - бутстрап может быть шумным при малом числе итераций (`B_BOOT`),
  - χ² зависит от биннинга (теряем часть информации),
  - Mood’s median test использует только информацию “выше/ниже медианы”, и может вести себя иначе, чем бутстрап по разности медиан.


---

## **Пункт 4. Какой подход мощнее в конкретном случае и почему**

Сравниваем “классические тесты” vs “бутстрап-проверку” (и немного — какие статистики вообще сильнее).

**В этом кейсе (очень большие выборки пользователей):**
- Асимптотические тесты (Mood’s median test, KS, χ²) обычно **мощные и стабильные**: p-value считается быстро и точно.
- Бутстрап “под H0” при достаточном числе итераций даёт близкий результат, но:
  - требует больше вычислений,
  - добавляет Monte-Carlo погрешность (особенно при малом `B_BOOT`).

**По мощности статистик:**
- Для “различия типичного уровня” часто мощнее тесты, использующие больше информации (например, Mann–Whitney), чем строгий тест медиан (Mood’s median test).
- Однако в задании просили гипотезы именно про **медианы**, поэтому Mood’s median test — самый прямой.

Ниже — быстрая *эмпирическая* иллюстрация: как часто тест отвергает H0 на подвыборках фиксированного размера (это не обязательно, но помогает аргументировать).


In [ ]:
def estimate_reject_rate(x_full, y_full, pvalue_fn, n=400, n_trials=60, alpha=0.05, seed=7):
    rng = np.random.default_rng(seed)
    rej = 0
    for _ in range(n_trials):
        xs = rng.choice(x_full, size=n, replace=True)
        ys = rng.choice(y_full, size=n, replace=True)
        p = pvalue_fn(xs, ys)
        if p < alpha:
            rej += 1
    return rej / n_trials

# p-value через Mood's median test
def p_median_test(a, b):
    _, p, _, _ = stats.median_test(a, b, ties="below")
    return p

# p-value через null bootstrap по разности медиан (меньше B для скорости)
def p_bootstrap_median(a, b):
    _, p, _ = null_bootstrap_pvalue(a, b, stat_median_diff, B=200, seed=RANDOM_STATE)
    return p

rate_median_test = estimate_reject_rate(x, y, p_median_test, n=350, n_trials=40, alpha=0.05, seed=1)
rate_bootstrap = estimate_reject_rate(x, y, p_bootstrap_median, n=350, n_trials=20, alpha=0.05, seed=2)

{"reject_rate_median_test_mean_rating": rate_median_test,
 "reject_rate_bootstrap_median_mean_rating": rate_bootstrap}


**Вывод:**  
- В больших выборках “классические” тесты обычно дают стабильные результаты и не требуют тяжелых вычислений.  
- Бутстрап хорош как проверка устойчивости, но при ограниченном числе итераций может быть шумнее.  
- Если сравнивать по мощности “под задачу”: тесты, использующие больше информации (ранги/вся CDF/частоты), чаще мощнее, чем тесты по одной характеристике.


---

## Дополнительно: коробка с усами


In [1]:
def boxplot_two_groups(valuesA, valuesB, title):
    plt.figure(figsize=(7,4))
    plt.boxplot([valuesA, valuesB], labels=[A, B], showfliers=False)
    plt.title(title)
    plt.grid(True, axis="y", alpha=0.3)
    plt.show()

boxplot_two_groups(x, y, "mean_rating (user-level)")
boxplot_two_groups(x_d, y_d, "unique_movies (user-level)")


NameError: name 'x' is not defined